In [1]:
import numpy as np t
from scipy import integrate as si 
from scipy.constants import c
from scipy import interpolate

#%matplotlib inline

In [2]:
'''
Calculator for calculating cosmological parameters such as Age of Universe and cosmological distances at different cosmic times
Written by: Branislav Avramov
Sept 2019

'''


#Python Calculator 
#Define all of the functions used by the calculator
#formulas mostly taken from: https://arxiv.org/abs/astro-ph/9905116  
def E_z(z, **cosmo):
    return (np.sqrt(cosmo['Omega_r']*(1+z)**4+cosmo['Omega_m']*(1+z)**3+cosmo['Omega_k']*(1+z)**2+cosmo['Omega_l']))

def hubble(z, **cosmo):

    return cosmo['H0']*E_z(z, **cosmo)

def omega(z, **cosmo):
    Omega_m = cosmo['Omega_m']*(1+z)**3/E_z(z, **cosmo)**2
    Omega_r = cosmo['Omega_r']*(1+z)**4/E_z(z, **cosmo)**2
    Omega_l = cosmo['Omega_l']/E_z(z, **cosmo)**2
    Omega_k = 1- cosmo['Omega_k'] - cosmo['Omega_l'] -cosmo['Omega_r'] -cosmo['Omega_m']
    return np.array([Omega_r, Omega_m, Omega_k, Omega_l])

def _comoving_integrand(zf, om_r, om_m, om_l, om_k, H):
    cosmo = {'Omega_m': om_m, 'Omega_r':om_r, 'Omega_l':om_l, 'Omega_k':om_k, 'H0': H}
    return (c/1e+3)/hubble(zf, **cosmo)

def comoving_integrand(z, **cosmo):
        return _comoving_integrand(z, cosmo['Omega_r'], cosmo['Omega_m'], cosmo['Omega_k'],cosmo['Omega_l'], cosmo['H0']) 

def comoving_dist(z, **cosmo):
 #si.quad does not accept **kwargs, so this a (ineffiecent, but working) workaround 
    om_r = cosmo['Omega_r']
    om_m = cosmo['Omega_m']
    om_l = cosmo['Omega_l']
    om_k = cosmo['Omega_k']
    H = cosmo['H0']
    d_co,err = si.quad(_comoving_integrand, 0, z, args= (om_r, om_m, om_l, om_k, H))
    return d_co

def proper_distance(z, d_com):
    return d_com/(1+z)

def comoving_dist_transverse(d_com, **cosmo):
    if (cosmo['Omega_k'] != 0):
        print('Not a flat Universe!')
    else:
        return d_com
    
def angular_dist(z, d_comt):
    return d_comt/(1+z)

def luminosity_dist(z, d_ang):
    return d_ang*(1+z)**2

def _lookback_integrand(z, om_r, om_m, om_l, om_k, H):
    cosmo = {'Omega_m': om_m, 'Omega_r':om_r, 'Omega_l':om_l, 'Omega_k':om_k, 'H0': H}
    return 1./((1+z)*hubble(z, **cosmo)*1e-3)

def lookback_time(z, **cosmo):
    om_r = cosmo['Omega_r']
    om_m = cosmo['Omega_m']
    om_l = cosmo['Omega_l']
    om_k = cosmo['Omega_k']
    H = cosmo['H0']
    l_time,err = si.quad(_lookback_integrand, 0, z, args= (om_r, om_m, om_l, om_k, H))
    return l_time
               
def universe_age(z, **cosmo):
    full_lookback = lookback_time(np.Inf, **cosmo)
    tl = lookback_time(z, **cosmo)
    age = full_lookback-tl
    return age    
    
#main function of the calculator, given a cosmological redshift returns all cosmological parameters    
def cosmo_calc(z, **cosmo):
    H = hubble(z, **cosmo) 
    print('Hubble parameter: %.3f (km/s)Mpc^-1'% H)
    omegas = omega(z, **cosmo)
    print('Energy densities: Omega_rad = %.3f, Omega_matter = %.3f, Omega_k = %.3f, Omega_Lambda = %.3f '% (omegas[0], omegas[1], omegas[2], omegas[3]))
    com_dist = comoving_dist(z, **cosmo)
    print('Comoving distance: %.4f Mpc'% com_dist)
    prop_dist = proper_distance(z, com_dist)
    print('Proper distance: %.4f Mpc'% prop_dist)
    ang_dist = angular_dist(z, comoving_dist_transverse(com_dist, **cosmo))
    print('Angular distance: %.4f Mpc'% ang_dist)
    lum_dist = luminosity_dist(z, ang_dist)
    print('Luminosity distance: %.4f Mpc'% lum_dist)
    lk_time = lookback_time(z, **cosmo)
    print('Lookback time: %.8e Gyr'% lk_time)
    age = universe_age(z, **cosmo)
    print('Age of the universe: %.8e Gyr'% age)
    return H, omegas, com_dist, ang_dist, lum_dist, lk_time, age
    
    
def scale_f_to_z(a):
    return 1/a-1

def z_to_scale_f(z):
    return 1/(1+z)
    
def g(z, **cosmo):
    omegas = omega(z, **cosmo)
    g = 5/2*omegas[1]/(omegas[1]**(4/7)-omegas[3]+(1+omegas[1]/2)*(1+omegas[3]/70))
    return g
    
def growth(z, **cosmo):
    d =  1/(1+z)*g(z, **cosmo)/g(0, **cosmo)
    return d

def p_spectrum_normalization(**cosmo):
    om_b = cosmo['Omega_b']
    om_m = cosmo['Omega_m']
    ns = cosmo['ns']
    sigma8 = cosmo['sigma8']
    H = cosmo['H0']
    intt,err = si.quad(_normalization_integrand, 1e-6, np.inf, args= (H, sigma8, om_m, om_b, ns))
    return (sigma8**2*2*np.pi**2)/(intt)
    
def _normalization_integrand(k, H, sigma8, om_m, om_b, ns):
    cosmo2 = {'Omega_m': om_m, 'Omega_b':om_b, 'sigma8':sigma8, 'H0': H}
    w = window_f8(k, **cosmo2)

    t = transfer_f(k, **cosmo2)
    return k**(2+ns)*t**2*w
    
def window_f8(k , **cosmo):
    R = 8/(cosmo['H0']/100)

    return 3*(np.sin(k*R)-k*R*np.cos(k*R))/(k*R)**3

    
def window_f(k,R,tmp,  **cosmo):
    return 3*(np.sin(k*R)-k*R*np.cos(k*R))/(k*R)**3


def transfer_f(k, **cosmo):
    h = cosmo['H0']/100
    G = cosmo['Omega_m']*h*np.exp(-cosmo['Omega_b']-np.sqrt(2*h)*cosmo['Omega_b']/cosmo['Omega_m'])
    q = k/G
    T = np.log(1+2.34*q)/(2.34*q)*1/(1+3.89*q+(16.1*q)**2+(5.46*q)**3+(6.71*q)**4)**0.25
    return T

def power_spectrum(k, z, **cosmo):
    return growth(z, **cosmo)**2*p_spectrum_normalization(**cosmo)*k**cosmo['ns']*transfer_f(k, **cosmo)**2

def _mass_var_integrand(k,z, H, M,rho_m,sigma8, om_m, om_b,om_r, om_l, om_k,  ns):
    cosmo2 = {'Omega_m': om_m, 'Omega_r':om_r, 'Omega_l':om_l, 'Omega_k':om_k, 'H0': H, 'Omega_b': om_b, 'sigma8': sigma8,'ns': ns}
    return k**2*power_spectrum(k, z, **cosmo2)*window_f(k,M,rho_m,  **cosmo2)**2

def mass_variance(z, M, **cosmo):

    rho_c = 2.775*1e+11*(cosmo['H0']/100)**2 # present value of rho_crit in solar mass per mpc^3
    rho_m = cosmo['Omega_m']*rho_c 
        
    om_b = cosmo['Omega_b']
    om_m = cosmo['Omega_m']
    om_l = cosmo['Omega_l']
    om_r = cosmo['Omega_r']
    om_k = cosmo['Omega_k']
    ns = cosmo['ns']
    sigma8 = cosmo['sigma8']
    H = cosmo['H0']
    intt, err = si.quad(_mass_var_integrand, 1e-6, np.inf, args = (z, H, M,rho_m,  sigma8, om_m, om_b, om_r, om_l,om_k, ns))
    return 1/np.pi*np.sqrt(0.5*intt), rho_m
    
    

### Usage:
Use the cosmo calculator by calling cosmo_calc(z, cosmo_dict), where cosmo_dict is a dictionary specifying cosmological parameters in the following form:

In [3]:
cosmo = {'Omega_m': 0.3, 'Omega_r':1e-4, 'Omega_l':0.7, 'Omega_k':0, 'H0': 72, 'Omega_b': 0.0486, 'sigma8': 0.8,'ns': 1 } #specify cosmological model

cosmo_calc(0.2, **cosmo)

Hubble parameter: 79.481 (km/s)Mpc^-1
Energy densities: Omega_rad = 0.000, Omega_matter = 0.425, Omega_k = -0.000, Omega_Lambda = 0.574 
Comoving distance: 793.9836 Mpc
Proper distance: 661.6530 Mpc
Angular distance: 661.6530 Mpc
Luminosity distance: 952.7803 Mpc
Lookback time: 2.41796492e+00 Gyr
Age of the universe: 1.09658633e+01 Gyr


(79.481196230555057,
 array([  1.70161454e-04,   4.25403635e-01,  -1.00000000e-04,
          5.74426204e-01]),
 793.9835879262093,
 661.6529899385079,
 952.7803055114513,
 2.417964924284734,
 10.965863274477181)